Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 3


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
import category_encoders as ce

In [0]:
data = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

data.shape, test.shape

In [0]:
#train/validate split
train, val = train_test_split(data, random_state=42)

train.shape, val.shape, test.shape

In [0]:
#Define a function to wrangle train, validate, and test sets in the same way. 
#Clean outliers and engineer features.

def wrangle(X):
  """Wrangle train, validate, and test sets in the same way"""
    
  # Prevent SettingWithCopyWarning
  X = X.copy()
    
  # About 3% of the time, latitude has small values near zero,
  # outside Tanzania, so we'll treat these values like zero.
  X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
  # When columns have zeros and shouldn't, they are like null values.
  # So we will replace the zeros with nulls, and impute missing values later.
  #confirmed model is .2% worse when excluding population here
  cols_with_zeros = ['longitude', 'latitude','construction_year','population']
  for col in cols_with_zeros:
      X[col] = X[col].replace(0, np.nan)
  
  #drop duplicate columns
  # quantity & quantity_group are duplicates, so drop one
  # payment == payment_type
  X = X.drop(columns=['quantity_group','payment_type'])
  
  #recode permit to 1/0
  #this doesn't turn out make a meaningful difference
  #X['permit'] = X['permit'].replace({True: 1, False: 0})
  
  #new feature: whether amount_tsh is 0 or not
  #it turns out to be wildly unimportant!
  #(might be helpful on a regression; trees don't need it)
  #X['0_tsh'] = X['amount_tsh'] == 0
  
  #convert date_recorded to datetime
  X['date_recorded'] = X['date_recorded'].apply(pd.to_datetime)
  
  #create 'age' column, the number of years between construction and evaluation
  X['age'] = X['date_recorded'].apply(lambda d: d.year) - X['construction_year']
  
  #pare down high-cardinality variables prior to encoding
  #if a category has fewer than threshold observations, nan it
  #since we're ordinal encoding now, let's try removing this
  #removing it = modest validation accuracy increase
  
  #threshold = 600 #tweaking threshold value for best results
  #value_counts = X['funder'].value_counts()
  #to_remove = value_counts[value_counts < threshold].index
  #X['funder'] = X['funder'].replace(to_remove, np.nan)
  
  #value_counts = X['installer'].value_counts()
  #to_remove = value_counts[value_counts < threshold].index
  #X['installer'] = X['installer'].replace(to_remove, np.nan)
  
  #value_counts = X['lga'].value_counts()
  #to_remove = value_counts[value_counts < threshold].index
  #X['lga'] = X['lga'].replace(to_remove, np.nan)
  
  #scheme_management has fine cardinality but a pesky "None" observed once
  X['scheme_management'] = X['scheme_management'].replace('None', np.nan)
  
  #bools for whether the waterpoint is marked as rural or urban
  #...make it a tiny bit worse
  #X['rural'] = X['lga'].apply(lambda s: "Rural" in s)
  #X['urban'] = X['lga'].apply(lambda s: "Urban" in s)
  
  #fill nans in latitude and longitude with averages for the region/basin
  #instead of relying on the simple imputer
  #using region makes the model worse by .3%
  #using basin makes it better by about the same amount!
  X2 = pd.DataFrame(columns=X.columns)

  for basin in X['basin'].value_counts().index:
    df = X[X['basin'] == basin]
    lat_mean = df['latitude'].mean()
    long_mean = df['longitude'].mean()
    df['latitude'] = df['latitude'].replace(np.nan,lat_mean)
    df['longitude'] = df['longitude'].replace(np.nan,long_mean)
    X2 = X2.append(df)
    
  # return the wrangled dataframe
  #return X
  return X2

train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

pd.options.display.max_columns = 100
train.head()

In [0]:
target = 'status_group'
features = train.columns.drop([target, 'date_recorded'])

X_train = train[features]
y_train = train[target]

oe = ce.OrdinalEncoder()
y_train_encoded = oe.fit_transform(y_train)

X_val = val[features]
y_val = val[target]

X_test = test[features]

In [0]:
#pipeline includes:
#encoding of categorical variables as ordinal/numeric
#random forest classifier
pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(random_state=42, n_jobs=-1)
)

#dict of parameters to try
param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'], #median
    'randomforestclassifier__n_estimators': range(100,500,25) #100
    'randomforestclassifier__min_samples_leaf': range(1,100,5), #71
    'randomforestclassifier__max_depth': range(3,20) #16
}

#make the randomized search
#oh my GOD this takes FOREVER
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=100, 
    cv=3, 
    scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1,
    refit=True
)

search.fit(X_train, y_train_encoded);

#search.fit(X_train, y_train);
#pipeline.fit(X_train, y_train)
#print('Validation Accuracy', pipeline.score(X_val, y_val))

In [0]:
#looking at the hyperparameters
print(pipeline.named_steps['randomforestclassifier'])

In [0]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

In [0]:
#pipeline using the best hyperparameters from the random search
pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median'), 
    RandomForestClassifier(max_depth=16, min_samples_leaf=71, 
                           n_estimators=100, random_state=42, n_jobs=-1)
)

pipeline.fit(X_train, y_train)

#hmmm way worse than yesterday! not a fan
print('Validation Accuracy', pipeline.score(X_val, y_val))